In [1]:
import os
import torchaudio
from speechbrain.pretrained import EncoderClassifier
import pandas as pd
from tqdm import tqdm

/home/nuttawac/.conda/envs/test_gender/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
torchvision is not available - cannot save figures


In [2]:
audio_path = "common_voice_th_25883229.wav"

classifier = EncoderClassifier.from_hparams(
    source="./pretrained_model", 
    hparams_file="./hparams_inference.yaml", 
    savedir="./save"
    )
signal, fs = torchaudio.load(audio_path) 
output_probs, score, index, class_pred = classifier.classify_batch(signal)


In [3]:
class_pred[0]

'Male'

## Inference whole dataset

In [4]:
df_path = "../data/commonvoice11/annotation/validated.tsv"

df = pd.read_csv(df_path, sep="\t")

df.head()

/tmp/ipykernel_1720847/1803988364.py:3: DtypeWarning: Columns (7,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(df_path, sep="\t")


,client_id,path,sentence,up_votes,down_votes,age,gender,accents,locale,segment
0,00086f5dc46f9038f13bbd829c4118fab3ac28688d5ee8...,common_voice_th_25695281.mp3,ใครเป็นผู้รับ,2,0,NaN,NaN,NaN,th,NaN
1,00245363f02e6f86a15de6793dc282d6f7095a8db6889b...,common_voice_th_31277118.mp3,รู้ได้ไงว่าเขาไม่หนุก,2,0,NaN,NaN,NaN,th,NaN
2,005237750a4d82bf743fe1e6eb0e4a579f0eb6a2455aaf...,common_voice_th_25722081.mp3,การที่จะทำเค้กแต่งงานชั้นเลิศคุณจะต้องใช้น้ำตา...,3,0,NaN,NaN,NaN,th,NaN
3,00abfc84881a0c3f6166adb1549b4c17085df70b833144...,common_voice_th_25792357.mp3,เขาหยุด แล้วคว้ามือออกมา เฉียดตัวฉันไปเส้นยาแเ...,2,0,NaN,NaN,NaN,th,NaN
4,00d8e38f434aecc821a4b83ca441d1e780de1e71cae734...,common_voice_th_25695038.mp3,ผู้ชายคือช้างเท้าหน้า แต่ผู้หญิงคือควาญช้าง,2,1,NaN,NaN,NaN,th,NaN


In [6]:
# ds_root = "/home/nuttawac/lab_dataset/commonvoice11/data/clips_wav"
ds_root = "../data/commonvoice11/clips/clips_wav"

gender_pred = []

for i, row in tqdm(df.iterrows(), total=df.shape[0]):
    audio_path = os.path.join(ds_root, row["path"].replace(".mp3", ".wav"))
    signal, fs = torchaudio.load(audio_path) 
    output_probs, score, index, class_pred = classifier.classify_batch(signal)
    pred_score = output_probs.numpy().tolist()[0]

    gender_pred.append(class_pred[0].lower())

df["gender pred"] = gender_pred

100%|██████████| 135897/135897 [16:13<00:00, 139.65it/s]


In [7]:
df.to_csv("./models/cv11_validated_pred.csv", index=False)

## Assign gender to each client_id

In [8]:
def _get_gender(sub_df, col_name):

    gender_counts = sub_df[col_name].value_counts()
    gender = gender_counts.idxmax()
    
    if gender_counts.shape[0] > 1:
        if gender_counts["male"] == gender_counts["female"]:
            gender = "Cant decide"

    return gender

def assign_gender(df):
    export_dict = {
        "client_id": [],
        # "gender": [],
        "gender pred": []
    }

    df = df.set_index("client_id")
    for i in df.index.unique().tolist():
        sub_df = df.loc[[i]]
        # gt_gender = _get_gender(sub_df, "gender")
        pred_gender = _get_gender(sub_df, "gender pred")

        export_dict["client_id"].append(i)
        # export_dict["gender"].append(gt_gender)
        export_dict["gender pred"].append(pred_gender)

    return pd.DataFrame(export_dict)

In [9]:
person_level_gender = assign_gender(df)

person_level_gender.head()

,client_id,gender pred
0,00086f5dc46f9038f13bbd829c4118fab3ac28688d5ee8...,male
1,00245363f02e6f86a15de6793dc282d6f7095a8db6889b...,female
2,005237750a4d82bf743fe1e6eb0e4a579f0eb6a2455aaf...,female
3,00abfc84881a0c3f6166adb1549b4c17085df70b833144...,male
4,00d8e38f434aecc821a4b83ca441d1e780de1e71cae734...,male


In [10]:
person_level_gender.to_csv("./models/cv11_gender_prediction.csv")